In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
from reproject import reproject_interp
from astropy import stats
import os 
from astropy.table import QTable, join, vstack, hstack, Column


plt.style.use('paper.mplstyle')

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/'

In [3]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(endswith):
                if '628e' in file: 
                    continue
                else: 
                    print(os.path.join(root, file))
                    file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))
    return(file_list)

files_maps = get_files('_hst_ha_err.fits', root_dir)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/ngc3627_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/ngc4254_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/ngc1512_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/ngc5068_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4321/hst_contsub/ngc4321_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1365n/hst_contsub/ngc1365n_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4303/hst_contsub/ngc4303_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1087/hst_contsub/ngc1087_hst_ha_err.fits
/Users/abarnes/Dropbox/work/Smallprojects/ga

In [11]:
output_stats = []

for file_maps in files_maps: 

    if '628e' in file_maps:
        continue
    if 'ngc628mosaic' in file_maps:
        continue

    # Loading files...
    hstha_hdu = fits.open(file_maps)[0]
    gal = file_maps.split(root_dir)[-1].split('/')[0]

    # Get RMS for whole map... 
    std = np.nanstd(hstha_hdu.data)
    mad_std = stats.mad_std(hstha_hdu.data, ignore_nan=True)
    mean = np.nanmean(hstha_hdu.data)
    median = np.nanmedian(hstha_hdu.data)
    try: 
        pix_scale = (np.absolute(hstha_hdu.header['CD1_1'])*u.deg).to('arcsec')
    except:
        pix_scale = (np.absolute(hstha_hdu.header['PC1_1'])*u.deg).to('arcsec')

    pix_area = pix_scale**2
    median_arcsec2 = (median/pix_area.value)/1e3 # erg/s/cm2/arcsec2 10-17

    output_stats += [[std, mean, median, median_arcsec2]]
    print("Galaxy:", gal, "STD:", std, "MAD_STD:", mad_std, "MEAN:", mean, "MEDIAN:", median, "MEDIAN_ARCSEC2:", median_arcsec2)

Galaxy: ngc3627 STD: 741.4372 MAD_STD: 11.202508066993124 MEAN: 135.07921 MEDIAN: 128.13225 MEDIAN_ARCSEC2: 81.6261857183995
Galaxy: ngc4254 STD: 860.69324 MAD_STD: 14.473515047471269 MEAN: 158.96504 MEDIAN: 149.96098 MEDIAN_ARCSEC2: 95.53210272728623
Galaxy: ngc1512 STD: 1109.9398 MAD_STD: 9.175863498454989 MEAN: 133.11783 MEDIAN: 121.093056 MEDIAN_ARCSEC2: 77.14189375360081
Galaxy: ngc1433 STD: 828.7947 MAD_STD: 11.035891774584465 MEAN: 128.91461 MEDIAN: 118.83138 MEDIAN_ARCSEC2: 75.70110315505939
Galaxy: ngc5068 STD: 854.72504 MAD_STD: 11.256813893188232 MEAN: 142.12285 MEDIAN: 138.38815 MEDIAN_ARCSEC2: 88.15967305007479
Galaxy: ngc4321 STD: 1178.9353 MAD_STD: 14.1879711448532 MEAN: 184.88867 MEDIAN: 172.38557 MEDIAN_ARCSEC2: 109.81760746579995
Galaxy: ngc1365n STD: 802.20074 MAD_STD: 10.236767007029353 MEAN: 117.70629 MEDIAN: 108.017845 MEDIAN_ARCSEC2: 68.81237808768068
Galaxy: ngc4303 STD: 1114.0188 MAD_STD: 11.439628049190182 MEAN: 147.83669 MEDIAN: 137.48105 MEDIAN_ARCSEC2: 87.5

In [12]:
filters = [
            ['IC~5332', 'F658N WFC3'],
            ['NGC~0628', 'F658N ACS'],
            ['NGC~1087', 'F657N WFC3'],
            ['NGC~1300', 'F658N ACS'],
            ['NGC~1300', 'F658N ACS'],
            ['NGC~1365', 'F657N WFC3'],
            ['NGC~1385', 'F657N WFC3'],
            ['NGC~1433', 'F657N WFC3'],
            ['NGC~1512', 'F658N WFC3'],
            ['NGC~1566', 'F658N WFC3'],
            ['NGC~1672', 'F658N ACS'],
            ['NGC~2835', 'F658N WFC3'],
            ['NGC~3351', 'F658N WFC3'],
            ['NGC~3627', 'F658N WFC3'],
            ['NGC~4254', 'F657N WFC3'],
            ['NGC~4303', 'F658N WFC3'],
            ['NGC~4321', 'F657N WFC3'],
            ['NGC~5068', 'F658N WFC3'],
            ['NGC~7496', 'F657N WFC3'],
            ]

filters = np.array(filters)

table_filters = QTable(filters, names=('gal_name', 'filter'))
table_filters

gal_name,filter
str10,str10
IC~5332,F658N WFC3
NGC~0628,F658N ACS
NGC~1087,F657N WFC3
NGC~1300,F658N ACS
NGC~1300,F658N ACS
NGC~1365,F657N WFC3
NGC~1385,F657N WFC3
NGC~1433,F657N WFC3
NGC~1512,F658N WFC3


In [14]:
# Get into a astropy table the output of the stats of the maps to .tex
gals = []
for file_maps in files_maps:
    gal = file_maps.split(root_dir)[-1].split('/')[0]
    gals += [gal]

output_stats = np.array([output_stats][0], dtype=np.float64)
gals = np.array(gals)

for i, gal in enumerate(gals):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    if '628C' in gal:
        gal_split = gal.split('628C')
        gal = gal_split[0] + '0628'
    gals[i] = gal

table1 = Column(gals, name='gal_name')
table2 = QTable(output_stats, names=('std', 'mean', 'median', 'median_arcsec2'))

table = hstack([table1, table2])
table.sort('gal_name')
table

gal_name,std,mean,median,median_arcsec2
str12,float64,float64,float64,float64
IC~,3295.718505859375,111.63263702392578,102.90373992919922,65.55445582682734
IC~5332,923.4144897460938,135.8402099609375,125.46607208251953,79.92770802847404
NGC~0628,170.98167419433594,58.40823745727539,57.81412887573242,36.830281971853665
NGC~1087,937.1162719726562,128.89627075195312,118.28903198242188,75.35560055664718
NGC~1300,1062.343505859375,128.21490478515625,117.80318450927734,75.04609317879344
NGC~1365N,802.2007446289062,117.70629119873047,108.0178451538086,68.81237808768068
NGC~1385,1057.1390380859375,156.63844299316406,146.50949096679688,93.33334201849144
NGC~1433,828.794677734375,128.91461181640625,118.83138275146484,75.70110315505939
NGC~1512,1109.9398193359375,133.11782836914062,121.09305572509766,77.14189375360081


In [15]:
table = join(table_filters, table, keys='gal_name')
table

gal_name,filter,std,mean,median,median_arcsec2
str12,str10,float64,float64,float64,float64
IC~5332,F658N WFC3,923.4144897460938,135.8402099609375,125.46607208251953,79.92770802847404
NGC~0628,F658N ACS,170.98167419433594,58.40823745727539,57.81412887573242,36.830281971853665
NGC~1087,F657N WFC3,937.1162719726562,128.89627075195312,118.28903198242188,75.35560055664718
NGC~1300,F658N ACS,1062.343505859375,128.21490478515625,117.80318450927734,75.04609317879344
NGC~1300,F658N ACS,1062.343505859375,128.21490478515625,117.80318450927734,75.04609317879344
NGC~1385,F657N WFC3,1057.1390380859375,156.63844299316406,146.50949096679688,93.33334201849144
NGC~1433,F657N WFC3,828.794677734375,128.91461181640625,118.83138275146484,75.70110315505939
NGC~1512,F658N WFC3,1109.9398193359375,133.11782836914062,121.09305572509766,77.14189375360081
NGC~1566,F658N WFC3,--,--,--,--


In [9]:
table_out = table['gal_name', 'filter', 'median_arcsec2']

formats={
        'gal_name': '%s',
        'filter': '%s',
        'median_arcsec2': '%0.1f'}

table_out.write('./tabs/galmap_errprops.tex', formats=formats, overwrite=True)

In [10]:
# Reorganise to match names in .fits files
gals = table['gal_name']

# for i, gal in enumerate(gals):
#     if '~' in gal:
#         gal_split = gal.split('~')
#         gal = gal_split[0] + gal_split[1]
#     table['gal_name'][i] = gal

table.write('./tabs/galmap_props.fits', overwrite=True)
table

gal_name,filter,std,mean,median,median_arcsec2
str12,str10,float64,float64,float64,float64
IC~5332,F658N WFC3,754.5157470703125,122.24543762207031,115.24809265136719,73.41838113987143
NGC~0628,F658N ACS,76.00861358642578,67.96708679199219,64.96614074707031,41.38644530095273
NGC~1087,F657N WFC3,937.1162719726562,128.89627075195312,118.28903198242188,75.35560055664718
NGC~1300,F658N ACS,133.32656860351562,43.37991714477539,41.482269287109375,26.426129812671125
NGC~1300,F658N ACS,133.32656860351562,43.37991714477539,41.482269287109375,26.426129812671125
NGC~1385,F657N WFC3,943.4440307617188,126.00540924072266,116.25623321533203,74.06061344467528
NGC~1433,F657N WFC3,828.794677734375,128.91461181640625,118.83138275146484,75.70110315505939
NGC~1512,F658N WFC3,1109.9398193359375,133.11782836914062,121.09305572509766,77.14189375360081
NGC~1566,F658N WFC3,1062.343505859375,128.21490478515625,117.80318450927734,75.04609317879344
